In [1]:
# Rede Neural convolucional simples para o problema de reconhecimento de dígitos (MNIST)
import tensorflow as tf
import random
# import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\Marcos\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# Parâmetros de aprendizagem
taxa_aprendizado = 0.001
quantidade_maxima_epocas = 500
batch_size = 100

In [3]:
# entrada dos place holders
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])   # imagem 28x28x1 (preto e branca)
Y = tf.placeholder(tf.float32, [None, 10])

In [4]:
# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
'''

'\nTensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)\nTensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)\nTensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)\n'

In [5]:
# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
'''

'\nTensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)\nTensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)\nTensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)\nTensor("Reshape_1:0", shape=(?, 3136), dtype=float32)\n'

In [6]:
# Classificador - Camada Fully Connected entrada 7x7x64 -> 10 saídas
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

In [7]:
# Define a função de custo e o método de otimização 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=taxa_aprendizado).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
# inicializa
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
# treina a rede
print('Rede inicialiada. Treinamento inicializado. Tome um cafe...')
for epoca in range(quantidade_maxima_epocas):
    custo_medio = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        custo_medio += c / total_batch

    print('Epoca:', '%04d' % (epoca + 1), 'perda =', '{:.9f}'.format(custo_medio))

print('Treinamento finalizado!')

Rede inicialiada. Treinamento inicializado. Tome um cafe...
Epoca: 0001 perda = 0.345585191
Epoca: 0002 perda = 0.091812595
Epoca: 0003 perda = 0.068328802
Epoca: 0004 perda = 0.056317300
Epoca: 0005 perda = 0.047015345
Epoca: 0006 perda = 0.041059294
Epoca: 0007 perda = 0.036586180
Epoca: 0008 perda = 0.032691747
Epoca: 0009 perda = 0.028204335
Epoca: 0010 perda = 0.025165824
Epoca: 0011 perda = 0.022292004
Epoca: 0012 perda = 0.020693827
Epoca: 0013 perda = 0.017078174
Epoca: 0014 perda = 0.015808559
Epoca: 0015 perda = 0.013600444
Epoca: 0016 perda = 0.011965570
Epoca: 0017 perda = 0.010958715
Epoca: 0018 perda = 0.009401810
Epoca: 0019 perda = 0.009381979
Epoca: 0020 perda = 0.007564330
Epoca: 0021 perda = 0.007028039
Epoca: 0022 perda = 0.007245969
Epoca: 0023 perda = 0.005022357
Epoca: 0024 perda = 0.004163541
Epoca: 0025 perda = 0.006280980
Epoca: 0026 perda = 0.004233844
Epoca: 0027 perda = 0.003410145
Epoca: 0028 perda = 0.005130775
Epoca: 0029 perda = 0.003957029
Epoca: 0030 

In [ ]:
# Teste o modelo e verifica a taxa de acerto
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Taxa de acerto:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))